# Работа с файловой системой

Для начала вспомним модуль `os`

In [1]:
import os.path
os.path.join("/tmp/1", "temp.file")  # конкатенация путей

'/tmp/1/temp.file'

In [4]:
import os
os.path.join(os.getcwd(), "tmp.txt", 'tmp2.txt')

'/home/nbuser/library/3. File System/tmp.txt/tmp2.txt'

In [5]:
os.path.dirname("/tmp/1/temp.file")  # имя каталога по заданному полному пути

'/tmp/1'

In [7]:
os.path.dirname("/tmp/1/temp/txt")

'/tmp/1/temp'

In [8]:
os.path.basename("/tmp/1/temp.file")  # имя файла по заданному полному пути

'temp.file'

In [9]:
os.path.normpath("/tmp//2/../1/temp.file")  # нормализация пути

'/tmp/1/temp.file'

In [10]:
os.path.exists("/tmp/1/temp.file")  # существует ли путь?

False

## Файлы
Python по умолчанию достаточно просто работает с файлами операционной системы, в C-подобном стиле.
Перед работой с файлом надо его открыть с помощью команды `open`.
```python
f = open('path/to/file', 'filemode', encoding='utf8')
```

Результатом этой операции будет указатель на файл, в котором указатель текущей позиции поставлен на начало или конец файла.

Давайте по порядку разберем все аргументы:
- `path/to/file` путь к файлу может быть относительным или абсолютным.  
Можно указывать в Unix-style (path/to/file) или в Windows-style (path\\to\\file)
- `filemode` режим, в котором файл нужно открывать.  
Записывается в виде строки, состоит из следующих букв:
 - **r** – открыть на чтение (по умолчанию)
 - **w** – перезаписать и открыть на запись (если файла нет, то он создастся)
 - **x** – создать и открыть на запись (если уже есть – исключение)
 - **a** – открыть на дозапись (указатель будет поставлен в конец)
 - **t** – открыть в текстовом виде (по умолчанию)
 - **b** – открыть в бинарном виде
- `encoding` – указание, в какой кодировке файл записан (utf8, cp1251 и т.д.)

При открытии файла в режимах на запись ставится блокировка на уровне операционной системы

### Указатель на позицию
При открытии файла внутри него ставится указатель текущей позиции для чтения.  
При открытии в режиме чтения или записи указатель ставится на начало, в режиме **a** – в конец.

In [11]:
f = open('test.txt', 'w', encoding='utf8')

# указатель на текущую позицию
f.tell()  # 0

0

In [13]:
# Запишем в файл строку
f.write("This is a test string\n")

22

В качестве результата метод `write` возвращает количество записаных символов. Данные буферизуются, т.е. попасть на жесткий диск могут не сразу, если критично – после записи вызывайте `f.flush()` или закрывайте файл.  

Проверим, куда теперь указывает указатель

In [14]:
f.tell()

22

In [15]:
print(f.write("This is a new string\n"))
print(f.tell())

21
43


Каждая операция записи или чтения смещает указатель вправо на определенное количество символов. Попробуем проверить это для чтения. В режиме чтения можно открыть несколько параллельных сессий, т.е. одновременных доступов, в отличии от записи.  

Но для начала нужно закрыть файл

In [16]:
# обязательно нужно закрыть файл иначе он будет заблокирован ОС
f.close()

In [17]:
f = open('test.txt', 'r', encoding='utf8')
print(f.tell())  # указываем на начало

0


`f.read(n)` – операция, читающая с текущего места **n** символов, если файл открыт в `t` режиме, или **n** байт, если файл открыт в `b` режиме, и возвращающая прочитанную информацию.

In [18]:
print(f.read(10))
print(f.tell())

This is a 
10


После прочтения, указатель на содержимое остается на той позиции, где чтение закончилось.

Если **n** не указать, будет прочитано «от забора до обеда», т.е. от текущего места указателя до конца файла.

In [19]:
f.read()  # считали остаток файла

'test string\nThis is a new string\n'

In [20]:
f.tell()

43

Указатель можно перемещать вручную по файлу с помощью функции 
``` python
f.seek(offset[, from_what=0])
```
Аргумент `from_what`:
- 0 – с начала файла
- 1 – с того места, где мы сейчас (только для `b` режима)
- 2 – с конца файла (только для `b` режима)

In [21]:
f.seek(1) # на второй символ с начала

1

In [22]:
f.tell()

1

In [23]:
f.seek(-2, 2)  # на второй символ с конца, только с b флагом

UnsupportedOperation: can't do nonzero end-relative seeks

In [24]:
# обязательно закрываем файл
f.close()

---
Бинарный формат

---
### Чтение и запись построчно
Зачастую с файлами удобнее работать построчно, поэтому для этого есть отдельные методы

In [25]:
f = open('test.txt', 'a', encoding='utf8')  # открываем файл на дозапись

sequence = ["other string\n", "123\n", "test test\n"]
f.writelines(sequence) # берет строки из sequence и записывает в файл (без переносов)

f.close()

Метод `f.writelines(sequence)` не будет сам за вас дозаписывать символ конца строки. Поэтому при необходимости его нужно записывать вручную.  

Попробуем теперь построчно считать файл

In [26]:
f = open('test.txt', 'r', encoding='utf8')

print(f.readlines())  # считывает все строки в список и возвращает список

f.close()

['This is a test string\n', 'This is a new string\n', 'other string\n', '123\n', 'test test\n']


Метод `f.readline()` возвращает строку (символы от текущей позиции до символа переноса строки)

In [27]:
f = open('test.txt', 'r', encoding='utf8')

print(f.readline())
print(f.read(4))
print(f.readline())

f.close()

This is a test string

This
 is a new string



Объект файл является **итератором**, поэтому его можно использовать в цикле `for`

In [28]:
f = open('test.txt') # можно перечислять строки в файле
for line in f:
    print(line.strip())  # print(line, end='')
    
f.close()

This is a test string
This is a new string
other string
123
test test


In [30]:
f = open('test.txt', 'r', encoding='utf8')
print(f)

<_io.TextIOWrapper name='test.txt' mode='r' encoding='utf8'>


In [31]:
print(iter(f))  # берем итератор от итерируемого объекта

<_io.TextIOWrapper name='test.txt' mode='r' encoding='utf8'>


In [32]:
# указатель на файл сам является итератором
id(f) == id(iter(f))

True

---
### Менеджер контекста with
После работы с файлом его нужно освободить с помощью метода `close()`

Хэндлер данного файла освобождается для операционной системы (если файл был открыт для записи) и другие приложения могут получать к нему доступ.

Если не закрыть – будет висеть, пока не закроете Python или garbage collector не решит удалить его

Для явного указания места работы с файлом, а также чтобы не забывать закрыть файл после обработки, существует менеджер контекста `with`.  

In [35]:
# В блоке менеджера контекста открытый файл «жив» и с ним можно работать, при выходе из блока – файл закрывается.
with open("test.txt", 'rb') as f:
    a = f.read(10)
    b = f.read(23)

f.read(3)  # Error!

ValueError: read of closed file

In [ ]:
f.name

Менеджер контекста неявно вызывает закрытие файла после работы. Что освобождает вас от забот о том, закрыли ли вы файл или нет.  
Закрытие файла происходит при любом стечении обстоятельств, даже если внутри `with` будет ошибка

## Сериализация

**Сериализация** – процесс перевода какой-либо структуры данных в другой формат, более удобный для хранения и/или передачи.  

Основная задача сериализации – эффективно (т.е. обычно с наименьшим размером) сохранить или передать данные, при условии, что их можно десериализовать в исходные.  

Чаще всего это работа на низком уровне с байтами и различными структурами. Но не обязательно.


### JSON 
**JSON** – JavaScript Object Notation – текстовый формат обмена данными.
Легко читаем людьми, однозначно записывает данные, подходит для сериализации сложных структур. Много используется в вебе.

Пример JSON:
```JavaScript
{
    "firstName": "Иван",
    "lastName": "Иванов",
    "address": {
        "streetAddress": "Московское ш., 101, кв.101",
        "city": "Ленинград",
        "postalCode": "101101“
    },
    "phoneNumbers": ["812 123-1234", "916 123-4567“]
}
```

**Преимущества**:
- Человекочитаем – легко проверять данные и находить ошибки в реализации
- Текстовый формат, нативно поддерживается в http
- Лаконичен
- Широко распространен

**Недостатки**:
- Поддерживает ограниченное число типов данных (числа, строки, Boolean, None, list, dict)
- Нельзя задать рекурсивные структуры

---
JSON – тоже формат сериализации, при этом в вебе сплошь и рядом используемый.  
Для работы с ним используется модуль `json`.

In [36]:
# импортируем модуль JSON
import json

Основные методы:
- `json.dumps(obj)` – преобразует объект в строку формата JSON
- `json.loads(str)` – преобразует строку формата JSON в объект языка Python (выдает `ValueError`, если строка неверная)



#### Кодирование
Процесс, при котором берется python объект, а на выходе получается строка формата json

In [37]:
# Закодируем Python объект в json строку
json.dumps([1, 2, 3, {'4': 5, '6': 7}])

'[1, 2, 3, {"4": 5, "6": 7}]'

In [38]:
# Закодируем Python объект в json строку компактным способом
json.dumps([1, 2, 3, {'4': 5, '6': 7}], separators=(',', ':'))

'[1,2,3,{"4":5,"6":7}]'

In [39]:
# вывод в читабельном формате
print(json.dumps([1, 2, 3, {'4': 5, '6': 7}], indent=4))

[
    1,
    2,
    3,
    {
        "4": 5,
        "6": 7
    }
]


In [41]:
print(json.dumps([1, 2, 3, {'4': 5, '6': 7}], indent=10))

[
          1,
          2,
          3,
          {
                    "4": 5,
                    "6": 7
          }
]


In [43]:
# json не поддерживает `tuple`
print(json.dumps([(1, 2, 3), [4, 5, 6]], indent='\t'))
{
    'json': [
                [
                    1,
                    2,
                    3
                ],
                [
                    4,
                    5,
                    6
                ]
            ]
}

[
	[
		1,
		2,
		3
	],
	[
		4,
		5,
		6
	]
]


{'json': [[1, 2, 3], [4, 5, 6]]}

#### Декодирование
Процесс обратный кодированию, при котором берется строка json формата, а на выходе получается объект python.  

Имейте ввиду результат декодирования не всегда эквивалентен исходному объекту

In [53]:
def check_code_decode_json(src):
    json_str = json.dumps(src)
    python_obj = json.loads(json_str)

    print('src:', src)
    print('json:', json_str)
    print('result', python_obj)
    
    return src == python_obj

In [46]:
src = [1, 2, 3, {"4": 5, '6': 7}]
check_code_decode_json(src)

src: [1, 2, 3, {'4': 5, '6': 7}]
json: [1, 2, 3, {"4": 5, "6": 7}]
result [1, 2, 3, {'4': 5, '6': 7}]


True

In [47]:
# использование tuple
src = [(1, 2, 3)]
check_code_decode_json(src)

src: [(1, 2, 3)]
json: [[1, 2, 3]]
result [[1, 2, 3]]


False

In [55]:
# в json ключи приводятся к строке, а в неявном виде не декодируется назад
src = [{4: 5, 6: 7}]
check_code_decode_json(src)

src: [{4: 5, '4': 7}]
json: [{"4": 5, "4": 7}]
result [{'4': 7}]


False

In [56]:
# в json ключи приводятся к строке, а в неявном виде не декодируется назад
src = [{4: 5, "4": 7}]
check_code_decode_json(src)

src: [{4: 5, '4': 7}]
json: [{"4": 5, "4": 7}]
result [{'4': 7}]


False

Не путайте `dumps` с `dump`, а `loads` с `load`!  
Последние функции предназначены для работы с каким-либо источником (файлом, потоком данных и т.д.)

---
### Pickle
Сериализация в JSON полезна в мире веба, если вам нужно что-то сохранить на диск – используется бинарная сериализация.

Самый популярный модуль для бинарной сериализации – `Pickle`.

Сохраняет объекты в бинарный формат, который потом можно восстановить при другом запуске или на другом устройстве.

Вследствие развития получилось, что у `pickle` есть несколько протоколов. Сохранять и загружать нужно с одним и тем же, по умолчанию в Python3 протокол – 3.

Внутри модуля реализовано всего несколько методов:
- `pickle.dump(obj, file)` – сохраняет объект **obj** в файл **file**
- `pickle.dumps(obj)` – сериализует объект и возвращает набор байт (для передачи по сети, например)
- `pickle.load(file)` – читает файл и восстанавливает объект из файла
- `pickle.loads(bytes_object)` – читает последовательность байт и восстанавливает по ним объект


Что можно «запиклить»?
- None, True, False
- целые, дробные и комплексные числа
- строки, байты, байт-массивы
- кортежи, списки, множества и словари, содержащие только "picklable" объекты
- функции, определенные на верхнем уровне модуля (только используя `def`, не лямбда функции)
- встроенные функции языка Python, определенные на верхнем уровне модуля
- классы, определенные на верхнем уровеня модуля
- экземпляры таких классов, чей результат `__dict__` или `__getstate__()` – "picklable"


### Pickle vs JSON
JSON – формат текстовой сериализации (чаще всего utf-8), а `pickle` – формат бинарной сериализации

JSON человекочитаем, а `pickle` – нет

JSON не привязан к языку и часто используется вне экосистемы языка Python, в то время как `pickle` специфичен для языка Python

JSON (по умолчанию) может сериализовывать только часть встроенных типов языка Python и не может сериализовывать собственноручно написанные классы, `pickle` может сериализовывать множество различных типов объектов

Если вы хотите сохранять данные, которые потом будут обрабатывать на python – берите `pickle`. Есть сомнения, чем данные будут читать – берите другой тип сериализации (в байты и с помощью `struct`, например). 
Передача данных в вебе – JSON или XML (даже если на другой стороне тоже python и pickle доступен).

### Не только Pickle
В языке Python существуют и другие способы сериализации:

Если объект можно перевести в байт-массив – можно с помощью `struct` перевести и сохранить в файл (преимущества – можно распарсить на другом языке)

Если объект – `NumPy` массив, можно использовать `np.save`, `np.savetxt`, `np.savez`, `np.savez_compressed`

Для хранения больших файлов (астрономические данные, веса больших нейронных сетей и т.д.) используется формат `HDF5`. Работа с такими файлами в Python осуществляется с помощью библиотеки `h5py` и методов `create_dataset`, `File` и т.д.

Многие модули имеют собственные методы для сериализации данных (часто в основе – pickle или struct)